<a href="https://colab.research.google.com/github/Laxmanlee/Anime-Recoomendation-system/blob/main/filtering_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
#%%
anime = pd.read_csv("anime.csv")
rating = pd.read_csv("rating.csv")
#%%
anime.isnull().sum()


anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [12]:
anime.fillna({'rating':0},inplace=True)

In [13]:
top_anime_rating_count = rating.groupby(by='anime_id').count()['rating'].sort_values(ascending=False).head(10)
top_anime_rating_count = pd.DataFrame({'rating_count':top_anime_rating_count}).reset_index()
top_anime_rating = pd.merge(top_anime_rating_count,anime,on='anime_id')
top_anime_rating.head(5)

,anime_id,rating_count,name,genre,type,episodes,rating,members
0,1535,39340,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917
1,11757,30583,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100
2,16498,29584,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229
3,1575,27718,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25,8.83,715151
4,226,27506,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511


In [14]:
episodes_rating = anime[(anime['episodes']!='Unknown')&(anime['type']=='TV')].astype({'episodes':int})
episodes_rating['episodes'].describe()


count    3578.000000
mean       35.915595
std        80.640355
min         2.000000
25%        12.000000
50%        24.000000
75%        39.000000
max      1818.000000
Name: episodes, dtype: float64

In [15]:
user_rating_count = rating.groupby(by='user_id').count()['anime_id']
#%%
anime_rating_count = rating.groupby(by='anime_id').count()['rating'].reset_index().rename(columns={'rating':'rating_count'})
#%%
filtered_anime = anime_rating_count[anime_rating_count['rating_count']>250]
filtered_anime.head(5)

,anime_id,rating_count
0,1,15509
1,5,6927
2,6,11077
3,7,2629
4,8,413


In [16]:
user_rating_count = rating.groupby(by='user_id').count()['rating'].reset_index().rename(columns={'rating':'rating_count'})
#%%

filtered_user = user_rating_count[user_rating_count['rating_count']>100]
#%%
filtered_rating_anime = rating[rating['anime_id'].isin(filtered_anime['anime_id'])]
filtered_rating = filtered_rating_anime[filtered_rating_anime['user_id'].isin(filtered_user['user_id'])]
#%%
rating_matrix = filtered_rating.pivot_table(index='user_id',columns='anime_id',values='rating').fillna(0)
print(rating_matrix.shape)
rating_matrix.head()

(24676, 3318)


anime_id,1,5,6,7,8,15,16,17,18,19,...,33142,33161,33201,33222,33338,33421,33524,33558,34103,34240
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,8.0,0.0,0.0,6.0,0.0,6.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
user_anime = anime[anime['name']=='Naruto']
#%%
user_anime_ratings = rating_matrix[int(user_anime['anime_id'])]
#%%
correlated_anime = rating_matrix.corrwith(user_anime_ratings).reset_index().rename(columns={0:'Correlation'})
correlated_anime.head()

,anime_id,Correlation
0,1,0.072989
1,5,0.028334
2,6,0.128868
3,7,0.046833
4,8,0.068644


In [20]:
recommended_anime = correlated_anime.sort_values(by='Correlation',ascending=False).head(11).iloc[1:]
#%%
pd.merge(recommended_anime,anime,on='anime_id',how='left').drop(columns=['anime_id','Correlation'])

,name,genre,type,episodes,rating,members
0,Naruto: Shippuuden Movie 1,"Action, Adventure, Comedy, Fantasy, Shounen",Movie,1,7.43,108170
1,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
2,Naruto Movie 2: Dai Gekitotsu! Maboroshi no Ch...,"Adventure, Comedy, Drama, Fantasy, Shounen, Su...",Movie,1,6.99,97308
3,Naruto: Shippuuden Movie 2 - Kizuna,"Action, Martial Arts, Shounen, Supernatural",Movie,1,7.43,97818
4,Naruto Movie 3: Dai Koufun! Mikazuki Jima no A...,"Action, Adventure",Movie,1,6.95,91526
5,Bleach,"Action, Comedy, Shounen, Super Power, Supernat...",TV,366,7.95,624055
6,Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...,"Action, Comedy, Martial Arts, Shounen, Super P...",Movie,1,7.50,83515
7,Naruto: Shippuuden Movie 4 - The Lost Tower,"Action, Comedy, Martial Arts, Shounen, Super P...",Movie,1,7.53,84527
8,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917
9,Naruto: Shippuuden Movie 5 - Blood Prison,"Action, Adventure, Martial Arts, Mystery, Shou...",Movie,1,7.62,75660
